In [28]:
"""
    Debugging Notes: 
    
    To Do List:
    Debug position update function, functional: OK
    Debug velocity update function; functional: OK
    obj_func function: OK
    _compute_pbest function: Debug
    
    
    Need to fix _compute_pbest():
        Constraint handler cannot count 1s in particle parts, need to count
"""



# Import modules
import numpy as np

# Import sphere function as objective function
from pyswarms.utils.functions.single_obj import sphere as f

# Import backend modules
import pyswarms.backend as P
from pyswarms.backend.topology import Star
#import pyswarms as ps
#import pyswarms.discrete.binary as PSO

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
my_topology = Star() # The Topology Class
my_options = {'c1': 1, 'c2': 1, 'w': 1}
c1 = 1
c2 = 1
w = 1
my_swarm = P.create_swarm(n_particles=10, dimensions=9, binary=True, discrete=True, options=my_options) # The Swarm Class

#Initialize cost to array of inf for max value, to_append is manually set below
to_append = np.array([float('inf'), float('inf'), float('inf'),float('inf'), float('inf'), float('inf'),float('inf'), float('inf'), float('inf'), float('inf')])
my_swarm.pbest_cost = np.append(my_swarm.pbest_cost, to_append)
my_swarm.current_cost = np.append(my_swarm.current_cost, to_append)

#create temp storade for pbest cost
temp_pbest_cost = []
temp_pbest_cost = np.append(temp_pbest_cost, to_append)

#Printing Values for debugging
print('The following are the attributes of our swarm: {}'.format(my_swarm.__dict__.keys()))
print("# of particles:",my_swarm.n_particles)
print("dimensions:",my_swarm.dimensions)
print("pos \n",my_swarm.position)
print("velo \n",my_swarm.velocity)
print("current cost \n",my_swarm.current_cost)
print("pbest pos \n",my_swarm.pbest_pos)
print("pbest cost \n",my_swarm.pbest_cost)
print("best cost \n",my_swarm.best_cost)

print(my_swarm.position[0][0:3])
print(my_swarm.position[0][3:6])
print(my_swarm.position[0][6:9])
#print((my_swarm.position[0][0:3]).count(1))

The following are the attributes of our swarm: dict_keys(['position', 'velocity', 'n_particles', 'dimensions', 'options', 'pbest_pos', 'best_pos', 'pbest_cost', 'best_cost', 'current_cost'])
# of particles: 10
dimensions: 9
pos 
 [[1 0 1 1 0 0 1 0 1]
 [1 0 1 0 0 1 0 0 1]
 [1 1 1 1 1 0 1 1 1]
 [0 1 0 1 0 0 0 1 1]
 [1 0 0 1 0 1 1 1 0]
 [0 0 1 1 0 1 0 0 0]
 [0 1 0 1 0 0 0 1 1]
 [0 0 0 0 0 1 0 1 1]
 [0 1 1 1 0 1 1 0 1]
 [0 1 0 0 0 1 0 1 0]]
velo 
 [[0.06480107 0.34028418 0.80852589 0.44116728 0.16522837 0.06830719
  0.90775306 0.27959063 0.10635978]
 [0.13180347 0.16693572 0.96877976 0.01438735 0.97751503 0.68352928
  0.3110302  0.63789527 0.79411643]
 [0.46856622 0.52284176 0.98902512 0.07900009 0.20030357 0.73249786
  0.88691075 0.30209093 0.29438309]
 [0.86857432 0.09507565 0.21898365 0.50316802 0.72556222 0.49462028
  0.56410778 0.82013335 0.15620118]
 [0.36231619 0.29326871 0.17464607 0.94924409 0.4179337  0.5600061
  0.5241658  0.04879116 0.23699797]
 [0.48889148 0.58963198 0.4465689

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [30]:
#Position computation, BPSO computation for position

def _compute_position(swarm):
    for i in range(swarm.n_particles):                  
        for j in range(my_swarm.dimensions):
            if _sigmoid(swarm.velocity[i][j]) < np.random.uniform(low=0.0, high=1.0):
                swarm.position[i][j] = 1
            else:
                swarm.position[i][j] = 0   
    return swarm.position

def _sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [31]:
#Velocity computation, BPSO computation for velocity

def _compute_velocity(swarm,c1,c2,w):
    for i in range(my_swarm.n_particles):                  
        for j in range(my_swarm.dimensions):
            swarm.velocity[i][j] = w*swarm.velocity[i][j] + \
            [c1*np.random.uniform(low=0.0, high=1.0)*swarm.pbest_pos[i][j]]-[c1*np.random.uniform(low=0.0, high=1.0)*swarm.position[i][j]] + \
            [c2*np.random.uniform(low=0.0, high=1.0)*my_swarm.best_cost]-[c2*np.random.uniform(low=0.0, high=1.0)*swarm.position[i][j]]
    return swarm.velocity

In [40]:
#Taken from pyswarms.backend.operators
def _compute_pbest(swarm,temp_pbest_cost):
    try:
        # Infer dimensions from positions
        dimensions = swarm.dimensions
        
        # Create a 1-D and 2-D mask based from comparisons
        # Creating 1-D mask
        mask_cost = swarm.current_cost < temp_pbest_cost
        #print("maskcost:\n",mask_cost)
        
        # Insert constraints here
        cover0 = [0, 1, 1, 1, 1, 1, 1, 1, 1]
        cover1 = [1, 0, 1, 1, 1, 1, 1, 1, 1]
        cover2 = [1, 1, 0, 1, 1, 1, 1, 1, 1]
        cover3 = [1, 1, 1, 0, 1, 1, 1, 1, 1]
        cover4 = [1, 1, 1, 1, 0, 1, 1, 1, 1]
        cover5 = [1, 1, 1, 1, 1, 0, 1, 1, 1]
        cover6 = [1, 1, 1, 1, 1, 1, 0, 1, 1]
        cover7 = [1, 1, 1, 1, 1, 1, 1, 0, 1]
        cover8 = [0, 1, 1, 1, 1, 1, 1, 1, 0]
        
        """
        # If particle does not meet constraint, set mask value for that particle to 0
        if (swarm.position[0][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover0
        if (swarm.position[0][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover0
        if (swarm.position[0][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover0
        
        if (swarm.position[1][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover1
        if (swarm.position[1][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover1
        if (swarm.position[1][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover1
        
        if (swarm.position[2][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover2
        if (swarm.position[2][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover2
        if (swarm.position[2][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover2
            
        if (swarm.position[3][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover3
        if (swarm.position[3][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover3
        if (swarm.position[3][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover3
        
        if (swarm.position[4][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover4
        if (swarm.position[4][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover4
        if (swarm.position[4][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover4
        
        if (swarm.position[5][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover5
        if (swarm.position[5][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover5
        if (swarm.position[5][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover5
        
        if (swarm.position[6][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover6
        if (swarm.position[6][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover6
        if (swarm.position[6][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover6
        
        if (swarm.position[7][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover7
        if (swarm.position[7][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover7
        if (swarm.position[7][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover7
            
        if (swarm.position[8][0:3]).count(1) < 2:
            mask_cost = mask_cost and cover8
        if (swarm.position[8][3:6]).count(1) < 2:
            mask_cost = mask_cost and cover8
        if (swarm.position[8][6:9]).count(1) < 2:
            mask_cost = mask_cost and cover8
        """
        
        # Extension of 1-D mask to 2-D mask
        mask_pos = np.repeat(mask_cost[:, np.newaxis], dimensions, axis=1)
        #print("maskpos:\n",mask_pos)
        
        # Apply masks, copy into 2nd variable if 3rd variable fits conditions
        new_pbest_pos = np.where(~mask_pos, swarm.pbest_pos, swarm.position)
        new_pbest_cost = np.where(~mask_cost, temp_pbest_cost, swarm.current_cost)
        
        #print("swarm current pos:",swarm.position)
        #print("new swarm pbest pos:",new_pbest_pos)
        #print("new swarm pbest cost:",new_pbest_cost)
        
    except AttributeError:
        rep.logger.exception(
            "Please pass a Swarm class. You passed {}".format(type(swarm))
        )
        raise
    else:
        return (new_pbest_pos, new_pbest_cost)

In [41]:
#test function, 2D version, to be implemented per element (Element is 1D array)
def obj_func(x):
    #print(x[0:9])
    L1 = x[0:3]
    #print(L1)
    L2 = x[3:6]
    #print(L2)
    L3 = x[6:9]
    #print(L3)
    f = (L1[0] + L1[1] + L1[2])*5 + (L2[0] + L2[1] + L2[2])*10 + (L3[0] + L3[1] + L3[2])*15
    return f


In [42]:
#print("Initial Value current cost:\n",my_swarm.current_cost)
#print("Initial Value current pos:\n",my_swarm.position)
#print("Initial Value pbest pos:\n",my_swarm.pbest_pos)
#print("Initial Value pbest cost:\n",my_swarm.pbest_cost)

iterations = 100 # Set 100 iterations
for i in range(iterations):
    
    # Part 0: Compute current cost
    j=0
    for j in range(my_swarm.n_particles):
        my_swarm.current_cost[j] = obj_func(my_swarm.position[j][:]) # Compute current cost (using obj func)
        temp_pbest_cost[j] = obj_func(my_swarm.pbest_pos[j][:])  # Compute personal best cost (using obj func)
    
    #my_swarm.pbest_cost = f(my_swarm.pbest_pos) # Compute current cost (using built in sphere func)
    #print("-------------------------------")
    #print("Pbest pos:\n",my_swarm.pbest_pos)
    #print("Pbest cost:\n",my_swarm.pbest_cost)
    #print("Current pos:\n",my_swarm.position)
    #print("Current cost:\n", my_swarm.current_cost)
   
    #Update personal best pos/cost 
    #(Edit this part, only update pbest_cost if particle position is valid)
    my_swarm.pbest_pos, my_swarm.pbest_cost = _compute_pbest(my_swarm,temp_pbest_cost) # Update and store
    
    # Part 2: Update global best
    # Note that gbest computation is dependent on your topology
    if np.min(my_swarm.pbest_cost) < my_swarm.best_cost:
        my_swarm.best_pos, my_swarm.best_cost = my_topology.compute_gbest(my_swarm)

    # Let's print our output
    if i%20==0:
        print('Iteration: {} | my_swarm.best_cost: {:.4f}'.format(i+1, my_swarm.best_cost))

    # Part 3: Update position and velocity matrices
    # Note that position and velocity updates are dependent on your topology
    
    #Fix the velocity and position computation for Binary application
    
    my_swarm.velocity = _compute_velocity(my_swarm,c1,c2,w)
    my_swarm.position = _compute_position(my_swarm)   
    
print('The best cost found by our swarm is: {:.4f}'.format(my_swarm.best_cost))
print('The best position found by our swarm is: {}'.format(my_swarm.best_pos))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [35]:
#Sample solution for counting 1s (This fix is functional)
#Import modules
import numpy as np

testarr = [[1, 0, 1, 0, 1, 1, 0, 0, 1],[1, 1, 1, 0, 1, 1, 1, 0, 1],[0, 0, 1, 0, 1, 0, 0, 0, 1]]
print("test array:",testarr)
print("number of arrays:",len(testarr))
print("length of array:",len(testarr[0]))

i=0
j=0

for i in range(len(testarr)):
    j = testarr[i].count(1)
    print("The 1s in array %i is %i" %(i,j))
    j=0


test array: [[1, 0, 1, 0, 1, 1, 0, 0, 1], [1, 1, 1, 0, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 0, 0, 0, 1]]
number of arrays: 3
length of array: 9
The 1s in array 0 is 5
The 1s in array 1 is 7
The 1s in array 2 is 3


In [28]:
arr0 = [1,2,3,4,5]
arr1 = [6,7,8,9,10]
arr2 = [11,12,13,14,15]
totalarr = [arr0, arr1, arr2]

sum = totalarr[0] + totalarr[1] + totalarr[2]
print(sum)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [39]:
import numpy as np
y = np.array([1, 2, 2, 2, 2, 0, 2, 3, 3, 3, 0, 0, 2, 2, 0])
print(np.count_nonzero(y == 1))


1
